In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import argparse

In [2]:
# load and show file
images_folder_path = os.getcwd() + "\images\\" 

image_path = images_folder_path + "key1.jpg"

image = cv2.imread(image_path)
cv2.imshow("orginal", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
image_shape = image.shape
print(image.shape)

(2448, 3264, 3)


In [3]:
# use mouse to select origin point
def mouse_callback(event,x,y,flags,param):
    # grab references to the global variables
    global refPt, cropping
    if event == cv2.EVENT_LBUTTONDBLCLK:
        refPt = [(x, y)]
        print(refPt)


In [4]:
# scale image down to 1000 pixels
r = 1000.0/image.shape[1]
dim = (1000,int(image.shape[0] * r))
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
cv2.namedWindow("resized")
cv2.setMouseCallback("resized", mouse_callback)

while True:
    cv2.imshow("resized", resized)
    key = cv2.waitKey(1) & 0xFF
    
    # if the 'c' key is pressed, break from the loop
    if key == ord("c"):
        cv2.destroyAllWindows()
        break


[(494, 301)]
[(461, 360)]


In [24]:
# reduce reflections in image

hsv_image = cv2.cvtColor(resized, cv2.COLOR_RGB2HSV)
h, s, v = cv2.split(hsv_image)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
v = clahe.apply(v)

hsv_image = cv2.merge([h, s, v])
hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
cv2.imshow("reduced reflection", hsv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
# reduce shadow and background

# DID NOT WORK

# fgbg = cv2.createBackgroundSubtractorMOG2()

fgmask_image = fgbg.apply(hsv_image)



cv2.imshow("fgmask", fgmask_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
# extract key from image
gray_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2GRAY)
# cv2.imshow("gray image", gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

edges = cv2.Canny(gray_image,100,150)
cv2.imshow("edges", edges)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [5]:

blurred = cv2.pyrMeanShiftFiltering(resized, 3, 3)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
kernel = np.ones((2,2),np.uint8)

lower_range = np.array([150, 10, 10])
upper_range = np.array([180, 255, 255])
mask = cv2.inRange(hsv, lower_range, upper_range)

dilation = cv2.dilate(mask,kernel,iterations = 1)

closing = cv2.morphologyEx(dilation, cv2.MORPH_GRADIENT, kernel)
closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel)

#Getting the edge of morphology
edge = cv2.Canny(closing, 175, 175)
_, contours,hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Find the index of the largest contour
areas = [cv2.contourArea(c) for c in contours]
max_index = np.argmax(areas)
cnt=contours[max_index]

x,y,w,h = cv2.boundingRect(cnt)
cv2.rectangle(resized,(x,y),(x+w,y+h),(0,255,0),2)


cv2.imshow('threshold', resized)
cv2.imshow('edge', edge)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [ ]:
def main(argv):
    # [load_image]
    # Check number of arguments
    if len(argv) < 1:
        print ('Not enough parameters')
        print ('Usage:\nmorph_lines_detection.py < path_to_image >')
        return -1
    # Load the image
    src = cv.imread(argv[0], cv.IMREAD_COLOR)
    # Check if image is loaded fine
    if src is None:
        print ('Error opening image: ' + argv[0])
        return -1
    # Show source image
    cv.imshow("src", src)
    # [load_image]
    # [gray]
    # Transform source image to gray if it is not already
    if len(src.shape) != 2:
        gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    else:
        gray = src
    # Show gray image
    show_wait_destroy("gray", gray)
    # [gray]
    # [bin]
    # Apply adaptiveThreshold at the bitwise_not of gray, notice the ~ symbol
    gray = cv.bitwise_not(gray)
    bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, \
                                cv.THRESH_BINARY, 15, -2)
    # Show binary image
    show_wait_destroy("binary", bw)
    # [bin]
    # [init]
    # Create the images that will use to extract the horizontal and vertical lines
    horizontal = np.copy(bw)
    vertical = np.copy(bw)
    # [init]
    # [horiz]
    # Specify size on horizontal axis
    cols = horizontal.shape[1]
    horizontal_size = cols / 30
    # Create structure element for extracting horizontal lines through morphology operations
    horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 1))
    # Apply morphology operations
    horizontal = cv.erode(horizontal, horizontalStructure)
    horizontal = cv.dilate(horizontal, horizontalStructure)
    # Show extracted horizontal lines
    show_wait_destroy("horizontal", horizontal)
    # [horiz]
    # [vert]
    # Specify size on vertical axis
    rows = vertical.shape[0]
    verticalsize = rows / 30
    # Create structure element for extracting vertical lines through morphology operations
    verticalStructure = cv.getStructuringElement(cv.MORPH_RECT, (1, verticalsize))
    # Apply morphology operations
    vertical = cv.erode(vertical, verticalStructure)
    vertical = cv.dilate(vertical, verticalStructure)
    # Show extracted vertical lines
    show_wait_destroy("vertical", vertical)
    # [vert]
    # [smooth]
    # Inverse vertical image
    vertical = cv.bitwise_not(vertical)
    show_wait_destroy("vertical_bit", vertical)
    '''
    Extract edges and smooth image according to the logic
    1. extract edges
    2. dilate(edges)
    3. src.copyTo(smooth)
    4. blur smooth img
    5. smooth.copyTo(src, edges)
    '''
    # Step 1
    edges = cv.adaptiveThreshold(vertical, 255, cv.ADAPTIVE_THRESH_MEAN_C, \
                                cv.THRESH_BINARY, 3, -2)
    show_wait_destroy("edges", edges)
    # Step 2
    kernel = np.ones((2, 2), np.uint8)
    edges = cv.dilate(edges, kernel)
    show_wait_destroy("dilate", edges)
    # Step 3
    smooth = np.copy(vertical)
    # Step 4
    smooth = cv.blur(smooth, (2, 2))
    # Step 5
    (rows, cols) = np.where(edges != 0)
    vertical[rows, cols] = smooth[rows, cols]
    # Show final result
    show_wait_destroy("smooth - final", vertical)
    # [smooth]